<a href="https://colab.research.google.com/github/sedmonster/EmpathoBot/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
train_texts = train_data['message'].values
train_labels = train_data['label'].values  # 0 for ham, 1 for spam
test_texts = test_data['message'].values
test_labels = test_data['label'].values

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
test_sequences = tokenizer.texts_to_sequences(test_texts)

max_length = 50
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

print(f"Training data shape: {train_padded.shape}")
print(f"Testing data shape: {test_padded.shape}")
print(f"Vocabulary size: {len(tokenizer.word_index)}")

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Embedding(10000, 32, input_length=max_length),
    layers.LSTM(32, return_sequences=False),
    layers.Dense(24, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_padded, train_labels,
    epochs=10,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
test_loss, test_acc = model.evaluate(test_padded, test_labels, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
def predict_message(message):
    sequence = tokenizer.texts_to_sequences([message])
    padded = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')

    prob = model.predict(padded, verbose=0)[0][0]

    label = "spam" if prob > 0.5 else "ham"

    return [float(prob), label]

In [ ]:
test_messages = [
    "Congratulations! You've won a free ticket to Bahamas! Call now!",
    "Hey, are we still meeting for lunch tomorrow?",
    "URGENT! Your account has been compromised. Click here to secure it.",
    "Thanks for the update. See you then.",
    "Free entry to win £1000 weekly! Text WIN to 87121"
]

for msg in test_messages:
    prob, label = predict_message(msg)
    print(f"Message: {msg[:50]}...")
    print(f"Prediction: {label} (probability: {prob:.4f})\n")